In [5]:
import numpy as np
import shutil
import os
from os.path import join
import json
import torch
import torchvision
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import warnings
warnings.filterwarnings("ignore")

from torchDataUtils import create_dataset, ApplyTransform
from torchTrainUtils import train

torch.manual_seed(42)
print(f"torch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")

torch version: 2.0.0.dev20230223
torchvision version: 0.2.2


# Set Project Folder Path

In [3]:
# Define project/data directory/file paths
project_dir = os.getcwd()
data_dir_1 = join(project_dir, "dog-breeds-data", "images", "Images")
csv_file_1 = join(project_dir, "dog-breeds-data", "index.csv")
# data_dir_2 = join(project_dir, "dog-breeds-data-2", "classes")
# csv_file_2 = join(project_dir, "dog-breeds-data-2", "index.csv")
models_dir = "models"

print("project_dir:", '\t', project_dir)
print("\ndata_dir_1:", '\t', data_dir_1)
print("csv_file_1:", '\t', csv_file_1)
# print("\ndata_dir_2:", '\t', data_dir_2)
# print("csv_file_2:", '\t', csv_file_2)

project_dir: 	 /Users/PaulG/Desktop/Machine Learning/object-detection/dog-breeds

data_dir_1: 	 /Users/PaulG/Desktop/Machine Learning/object-detection/dog-breeds/dog-breeds-data/images/Images
csv_file_1: 	 /Users/PaulG/Desktop/Machine Learning/object-detection/dog-breeds/dog-breeds-data/index.csv


In [4]:
# define model hyperparameters
size = 400
n_epochs = 5
batch_size = 64
learning_rate = 0.001

# Create dataset from dataframe
full_dataset = create_dataset(project_dir=project_dir, csv_files=[csv_file_1])

# Define 80-20 split for train and test sets
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

# Perform train test split
print("\nSplit data into train and test sets")
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Apply any data augmentations to train and test sets
print("Apply data augmentations to train and test data")
train_dataset = ApplyTransform(dataset=train_dataset, size=size, transform="train")
test_dataset = ApplyTransform(dataset=test_dataset, size=size, transform="test")

print("Create train and test dataloaders")
train_dataloader = DataLoader(train_dataset, 
                              batch_size=batch_size, 
                              shuffle=True)
test_dataloader = DataLoader(test_dataset, 
                             batch_size=batch_size, 
                             shuffle=False)

classes = full_dataset.classes
num_classes = len(classes)

print(f"\n{num_classes} Classes:")
print(classes)



Split data into train and test sets
Apply data augmentations to train and test data
Create train and test dataloaders

120 Classes:
['affenpinscher', 'afghan_hound', 'african_hunting_dog', 'airedale', 'american_staffordshire_terrier', 'appenzeller', 'australian_terrier', 'basenji', 'basset', 'beagle', 'bedlington_terrier', 'bernese_mountain_dog', 'black-and-tan_coonhound', 'blenheim_spaniel', 'bloodhound', 'bluetick', 'border_collie', 'border_terrier', 'borzoi', 'boston_bull', 'bouvier_des_flandres', 'boxer', 'brabancon_griffon', 'briard', 'brittany_spaniel', 'bull_mastiff', 'cairn', 'cardigan', 'chesapeake_bay_retriever', 'chihuahua', 'chow', 'clumber', 'cocker_spaniel', 'collie', 'curly_coated_retriever', 'dandie_dinmont', 'dhole', 'dingo', 'doberman', 'english_foxhound', 'english_setter', 'english_springer', 'entlebucher', 'eskimo_dog', 'flat_coated_retriever', 'french_bulldog', 'german_shepherd', 'german_short-haired_pointer', 'giant_schnauzer', 'golden_retriever', 'gordon_setter'

In [7]:
# class Model(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()

#         # Load base model
#         self.model = torchvision.models.inception_v3(pretrained=True)
        
#         # Freeze base layers
#         for param in self.model.parameters():
#             param.requires_grad = False

#         self.model.fc = nn.Sequential(
#             nn.Linear(2048, num_classes)
#         )
    
#     def forward(self, x):
#         return self.model(x)

# Load base model
model = torchvision.models.inception_v3(pretrained=True)

# Freeze base layers
for param in model.parameters():
    param.requires_grad = False

# Remove fully connected layer
num_ftrs = model.fc.in_features
model.aux_logits = False
fc = nn.Sequential(
    nn.Linear(num_ftrs, num_classes)
)
model.fc = fc

mps_device = torch.device("mps")
model.to(mps_device)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [30]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

# Train model
history = train(model=model,
                input_size=size,
                classes=classes,
                train_dataloader=train_dataloader, 
                test_dataloader=test_dataloader,
                n_epochs=n_epochs,
                batch_size=batch_size,
                criterion=criterion, 
                optimizer=optimizer, 
                learning_rate=learning_rate,
                device=mps_device, 
                project_dir=project_dir,
                models_dir=models_dir)


Starting training phase for epoch 1/5
[Epoch: 1/5 Batch: 258/258] loss: 3.2102 accuracy: 38.4961 batch_accuracy: 31.2500
Starting validation phase for epoch 1/5
[Train loss: 3.2102],  [Test loss: 1.2368],  [Validation Accuracy: 84.79]
Saved model with accuracy 84.7911

Starting training phase for epoch 2/5
[Epoch: 2/5 Batch: 258/258] loss: 1.8973 accuracy: 56.6691 batch_accuracy: 31.2500
Starting validation phase for epoch 2/5
[Train loss: 1.8973],  [Test loss: 0.6383],  [Validation Accuracy: 88.68]
Saved model with accuracy 88.6783

Starting training phase for epoch 3/5
[Epoch: 3/5 Batch: 258/258] loss: 1.6186 accuracy: 60.4531 batch_accuracy: 75.0000
Starting validation phase for epoch 3/5
[Train loss: 1.6186],  [Test loss: 0.4874],  [Validation Accuracy: 89.82]
Saved model with accuracy 89.8202

Starting training phase for epoch 4/5
[Epoch: 4/5 Batch: 258/258] loss: 1.5199 accuracy: 61.4492 batch_accuracy: 56.2500
Starting validation phase for epoch 4/5
[Train loss: 1.5199],  [Test